In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from import_casa import casa

In [3]:
import pandas as pd
import pickle
from gensim.models import FastText

In [4]:
spm_centroids = pd.read_csv(casa.get_data_path() / "eda/spm/spm_centroids.csv", index_col=None)
with open(casa.get_data_path()/"eda/spaCy_pipeline/raw-data-cht-2020-ppline.pkl", "rb") as fin:
    data = pickle.load(fin)

In [5]:
spm_centroids

,Unnamed: 0,cluster_idx,cluster_size,centroid,words
0,0,274,19,bile,ay my line Line LINE Mobile riDay mobile Hami ...
1,1,118,21,中華電信,遠傳 中華電信 亞太 台灣之星 台灣大哥大 亞太電信 台灣大 中華電 遠傳的 遠傳電信 跟遠...
2,2,182,19,12pro,Pro 11 25 iphone pro 128 128G iphone12 12Pro a...
3,3,270,17,Pixel,支援 Pixel Google 顯示 心得 三星 google 小米 版本 pixel 4a...
4,4,192,31,12、iPhone,12 iPhone 預購 iPhone12 預約 新機 推出 開放 各 、遠傳 各家 的iP...
...,...,...,...,...,...
153,153,228,15,久,快 滿 久 多了 好用 後悔 鮮 夯 迅 豔 嚐 譜 挾 巔 冗
154,154,98,15,爭,代 取 利 屬 便 超商 爭 全家 索 宅配 暖 礙 潤 擷 侷
155,155,27,46,車站,區 台北 山 北 南 隔 離 島 車站 嘉 台南 高雄 台北市 湖 壁 蘭 桃園 橋 宜蘭 ...
156,156,303,36,樵,XD DD XDD 瞎 ㄌ 吹 辣 削 ㄛ ㄅ 賊 樁 😃 爹 奸 抹 拎 甩 賤 尛 錒 ...


In [6]:
centroid_lists = "中華電信,12pro,Pixel,1399元,24個月,199,LTE,3000,方案,5G,划算,電信業者,用中華,21M,攜碼".split(",")

In [7]:
path = casa.get_data_path() / "eda/spm/sp_vectors.model"
model = FastText.load(str(path))
model.wv.vectors.shape

[INFO] 2021-01-04 15:03:05,191 gensim.utils: loading FastText object from E:\LangOn\casa\data\eda\spm\sp_vectors.model
[INFO] 2021-01-04 15:03:05,262 gensim.utils: loading wv recursively from E:\LangOn\casa\data\eda\spm\sp_vectors.model.wv.* with mmap=None
[INFO] 2021-01-04 15:03:05,263 gensim.utils: loading vectors_ngrams from E:\LangOn\casa\data\eda\spm\sp_vectors.model.wv.vectors_ngrams.npy with mmap=None
[INFO] 2021-01-04 15:03:05,686 gensim.utils: setting ignored attribute vectors_ngrams_norm to None
[INFO] 2021-01-04 15:03:05,687 gensim.utils: setting ignored attribute vectors_norm to None
[INFO] 2021-01-04 15:03:05,688 gensim.utils: setting ignored attribute vectors_vocab_norm to None
[INFO] 2021-01-04 15:03:05,688 gensim.utils: setting ignored attribute buckets_word to None
[INFO] 2021-01-04 15:03:05,689 gensim.utils: loading vocabulary recursively from E:\LangOn\casa\data\eda\spm\sp_vectors.model.vocabulary.* with mmap=None
[INFO] 2021-01-04 15:03:05,689 gensim.utils: loading 

(5885, 100)

In [8]:
dep_data = data[0].main.title_tree
words = [x[0] for x in dep_data]    
seeds = spm_centroids.loc[spm_centroids.centroid=="方案"].iloc[0].words.split()
pots = casa.compute_potentials(words, seeds, model.wv)
targets, targ_indices = casa.extract_with_potentials(words, pots)   

## Sandboxing Tree eval

In [9]:
tree = casa.make_tree(dep_data)

In [24]:
' '.join([x[0] for x in dep_data])

'最近 各 家 電信 一直 打電話 傳 簡訊 詢問 要不要 續約    有夠 煩 的 但 畢竟 合約 有 30 個 月  而 5G 未來 會 普及 也 是 不變 事實 ( ? 想 問板 上 大大 覺得 要 續約 嗎 ？ 還是 等 合約 到期    直接續 488 學生 方案 呢 ？'

In [25]:
targets, targ_indices

(['續約', '合約', '續約', '合約', '488學生方案'], [[10], [17], [39], [44], [48, 49, 50]])

In [47]:
list(chain.from_iterable(targ_indices))

[10, 17, 39, 44, 48, 49, 50]

In [48]:
from itertools import chain
mask_ids = list(chain.from_iterable(targ_indices))
for site_ids in targ_indices:
    casa.find_eval_text(tree, site_ids, mask_ids)
    

合約 <有-dep>: 有 30 個 月  而 5G 未來 會 普及 也 是 不變 事實 ( ?
續約 <覺得-ccomp>: 問板 上 大大 覺得
合約 <到期-nsubj>: 到期   
488學生方案 <到期-dobj>: 到期   
488學生方案 <方案-dep/compound:nn>: 直接續 方案


In [92]:
from collections import Counter
from tqdm.auto import tqdm


def find_targets(dep_data, seeds, wv):
    try:
        words = [x[0] for x in dep_data]    
        pots = casa.compute_potentials(words, seeds, wv)
        targets, targ_indices = casa.extract_with_potentials(words, pots)        
    except Exception as ex:
        print(ex)
        targets = []
        targ_indices = []
    return targets, targ_indices

def process_tree(dep_data, tokens, seeds, wv):    
    targets, targ_indices = find_targets(dep_data, seeds, model.wv)
    mask_ids = list(chain.from_iterable(targ_indices))            
    eval_texts = []
    dep_tree = casa.make_tree(dep_data, tokens)
    for site_ids in targ_indices:
        eval_entries = casa.find_eval_text(dep_tree, site_ids, mask_ids)
        eval_texts.extend([x["evaltext"] for x in eval_entries])
    return targets, eval_texts

def extract_targets(data, seeds, centroid):
    target_list = []
    eval_texts = []
    for data_item in tqdm(data, desc=centroid):    
        # titles
        if data_item.main and data_item.main.title_tree:
            targets, evals = process_tree(
                data_item.main.title_tree, 
                data_item.main.proc_info["ckip"]["text"],
                seeds, model.wv)
            target_list.extend(targets)
            eval_texts.extend(evals)

        # replies
        for reply_x in data_item.replies:
            if not reply_x.text_tree: continue
            targets, evals = process_tree(
                reply_x.text_tree, 
                reply_x.proc_info["ckip"]["text"], 
                seeds, model.wv)
            target_list.extend(targets)
            eval_texts.extend(evals)
            
    return Counter(target_list), eval_texts

In [93]:
seeds = spm_centroids.loc[spm_centroids.centroid==centroid_lists[0]].iloc[0].words.split()

In [114]:
targets, eval_texts = extract_targets(data, seeds, centroid_lists[0])

In [115]:
targets.most_common(10)

[('5G', 7049),
 ('遠傳', 5726),
 ('中華', 4431),
 ('電信', 3468),
 ('亞太', 3285),
 ('台星', 3205),
 ('中華電信', 3181),
 ('4G', 2493),
 ('台哥大', 2176),
 ('大哥大', 1471)]

In [116]:
eval_counter = Counter(chain.from_iterable(eval_texts))

In [128]:
grams_counter = Counter()
for eval_text in eval_texts:
    for idx in range(len(eval_text)):
        if idx+2 > len(eval_text):
            break        
        tok = eval_text[idx:idx+2]
        if any("VH" in x[1] for x in tok):
            tok = "".join(f"{x[0]}({x[1]})" for x in tok)        
            grams_counter[tok] += 1

In [129]:
grams_counter.most_common(100)

[('到(DE)飽(VH)', 359),
 ('吃到(VC)飽(VH)', 357),
 ('最(Dfa)高(VH)', 157),
 ('飽(VH)，(COMMACATEGORY)', 137),
 ('不(D)限速(VH)', 123),
 ('飽(VH)的(DE)', 118),
 ('三(Neu)大(VH)', 103),
 ('限(VJ)速(VH)', 97),
 ('不(D)好(VH)', 72),
 ('就(D)好(VH)', 71),
 ('上市(VH)販售(VD)', 67),
 ('有用到(VH)才(Da)', 66),
 ('好(VH)，(COMMACATEGORY)', 65),
 ('比較(Dfa)好(VH)', 62),
 ('用戶(Na)升級(VH)', 62),
 ('速(VH)吃(VC)', 60),
 ('，(COMMACATEGORY)直接(VH)', 58),
 ('台灣(Nc)大(VH)', 57),
 ('及(Caa)上市(VH)', 55),
 ('最(Dfa)低(VH)', 53),
 ('多(VH)，(COMMACATEGORY)', 51),
 ('好(VH)  (WHITESPACE)', 49),
 ('飽(VH)  (WHITESPACE)', 45),
 ('更(Dfa)大(VH)', 44),
 ('合作(VH)，(COMMACATEGORY)', 44),
 ('最(Dfa)快(VH)', 42),
 ('夠(VH)用(VC)', 42),
 ('新(VH)的(DE)', 41),
 ('免費(VH)，(COMMACATEGORY)', 40),
 ('舊(VH)i(FW)', 40),
 ('直接(VH)在(P)', 40),
 ('速(VH)吃到(VC)', 39),
 ('很(Dfa)差(VH)', 39),
 ('很(Dfa)爛(VH)', 38),
 ('限量(VH)網路(Na)', 38),
 ('5(Neu)大(VH)', 38),
 ('老(VH)客戶(Na)', 37),
 ('很(Dfa)大(VH)', 36),
 ('好(VH)的(DE)', 36),
 ('升級(VH)速度(Na)', 36),
 ('7月(Nd)更多(VH)', 36),
 ('更多(VH)了(Di)', 3